# Search Engine

This lab is about starting from scratch, and hence having to structure the code yourself. A specification for the program you are to implement is given below - do pay attention as misunderstandings may cost you marks! Being precise is a necessary skill for a programmer. To give an executive summary, you are to code a search engine for recipes. A data set has been provided. The search engine is to be pretty basic, returning all recipes that contain all of the provided keywords. However, the user can choose from a number of orderings depending on their food preferences, which you need to support.

## Specification

The system must provide a function ``search``, with the following specification:
```
def search(query, ordering = 'normal', count = 10):
  ...
```

It `print`s out the results of the search, subject to the following rules:
1. It selects from the set of all recipes that contain __all__ of the words in the query (the positions/order of the words in the recipe are to be ignored).
2. It orders them based on the provided ordering (a string, meaning defined below).
3. It `print`s the top `count` matches only, preserving the order from best to worst. Must `print` just their title, one per line. Must be less than `count` if the search is specific enough that less than `count` recipes match.

As an aside, do not worry about memory usage. If duplicating some data can make your code faster/neater then feel free.



### Data set

A file, `recipes.json` is provided, containing 17K recipes. It can be parsed into a Python data structure using the [`json`](https://docs.python.org/3/library/json.html) module. It is a list, where each recipe is a dictionary containing various keys:
* `title` : Name of recipe; you can assume these are unique
* `categories` : A list of tags assigned to the recipe
* `ingredients` : What is in it, as a list
* `directions` : List of steps to make the recipe
* `rating` : A rating, out of 5, of how good it is
* `calories` : How many calories it has
* `protein` : How much protein is in it
* `fat` : How much fat is in it

Note that the data set was obtained via web scrapping and hence is noisy - every key except for `title` is missing from at least one recipe. Your code will need to cope with this.

You will probably want to explore the data before starting, so you have an idea of what your code has to deal with.

Data set came from https://www.kaggle.com/hugodarwood/epirecipes/version/2 though note it has been cleaned it up, by deleting duplicates and removing the really dodgy entries.



### Search

The search should check the following parts of the recipe (see data set description below):
* `title`
* `categories`
* `ingredients`
* `directions`

For instance, given the query "banana cheese" you would expect "Banana Layer Cake with Cream Cheese Frosting" in the results. Note that case is to be ignored ("banana" matches "Banana") and the words __do not__ have to be next to one another, in the same order as the search query or even in the same part of the recipe ("cheese" could appear in the title and "banana" in the ingredients). However, all words in the search query __must__ appear somewhere.



### Tokenisation

This is the term for breaking a sentence into each individual word (token). Traditionally done using regular expressions, and Python does have the `re` module, but there is no need to do that here (`re` can be quite fiddly). For matching words your tokenisation must follow the following steps:
1. Convert all punctuation and digits into spaces. For punctuation use the set in [`string.punctuation`](https://docs.python.org/3/library/string.html#string.punctuation), for digits [`string.digits`](https://docs.python.org/3/library/string.html#string.digits). You may find [`translate()`](https://docs.python.org/3/library/stdtypes.html#str.translate) interesting!
2. [`split()`](https://docs.python.org/3/library/stdtypes.html#str.split) to extract individual tokens.
3. Ignore any token that is less than $3$ characters long.
4. Make tokens lowercase.

When matching words for search (above) or ordering (below) it's only a match if you match an entire token. There are many scenarios where this simple approach will fail, but it's good enough for this exercise. The auto marker will be checking the above is followed! When doing a search the code should ignore terms in the search string that fail the above requirements.



### Ordering

There are three ordering modes to select from, each indicated by passing a string to the `search` function:
* `normal` - Based simply on the number of times the search terms appear in the recipe. A score is calculated and the order is highest to lowest. The score sums the following terms (repeated words are counted multiple times, i.e. "cheese cheese cheese" is $3$ matches to "cheese"):
    * $8 \times$ Number of times a query word appears in the title
    * $4 \times$ Number of times a query word appears in the categories
    * $2 \times$ Number of times a query word appears in the ingredients
    * $1 \times$ Number of times a query word appears in the directions
    * The `rating` of the recipe (if not available assume $0$)

* `simple` - Tries to minimise the complexity of the recipe, for someone who is in a rush. Orders to minimise the number of ingredients multiplied by the numbers of steps in the directions.

* `healthy` - Order from lowest to highest by this cost function:
$$\frac{|\texttt{calories} - 510n|}{510} + 2\frac{|\texttt{protein} - 18n|}{18} + 4\frac{|\texttt{fat} - 150n|}{150}$$
Where $n \in \mathbb{N}^+$ is selected to minimise the cost ($n$ is a positive integer and $n=0$ is not allowed). This can be understood in terms of the numbers $510$, $18$ and $150$ being a third of the recommended daily intake (three meals per day) for an average person, and $n$ being the number of whole meals the person gets out of cooking/making the recipe. So this tries to select recipes that neatly divide into a set of meals that are the right amount to consume for a healthy, balanced diet. Try not to overthink the optimisation of $n$, as it's really quite simple to do!

To clarify the use of the ordering string, to get something healthy that contains cheese you might call `search('cheese', 'healthy')`. In the case of a recipe that is missing a key in its dictionary the rules are different for each search mode:
* `normal` - Consider a missing entry in the recipe (e.g. no `ingredients` are provided) to simply mean that entry can't match any search words (because it has none!), but the item is still eligible for inclusion in the results, assuming it can match the search with a different entry.
* `simple` - If a recipe is missing either `ingredients` or `directions` it is dropped from such a search result. Because the data is messy if either of these lists is of length $1$ it should be assumed that the list extraction has failed and the recipe is to also be dropped from the search results.
* `healthy` - If any of `calories`, `protein` or `fat` is missing the recipe should be dropped from the result.



### Extra

You may find the [_inverted index_](https://en.wikipedia.org/wiki/Inverted_index) interesting. It's a data structure used by search engines. For each word a user may search for this contains a list of all documents (recipes) that contain the word. This may take a little effort to understand, but the resulting code will be both faster and neater.

## Advice

* Don't just start coding: Make a plan and work out what you intend to do.
* Think about structure, as messy code leads to mistakes.
* Plan your data structures. Don't be afraid to use sheets of paper if that works for you!
* Don't duplicate code, put it in a function/method instead.

* Divide and conquer. Break the system into parts that can implemented with minimal dependency on the rest. Functions or OOP are both suitable for doing this.
* Test early. Verify the individual parts work before trying to combine them. Factor this into the order you implement the parts of the system - don't implement something you are going to struggle to test before implementing, and verifying, other parts.
* Do not try and do a 'big bang', where you get it all working at once. Instead, get it working with features missing, then add those features in, one at a time.

* Keep things as simple as possible. Avoid long functions/methods.
* Include comments, as a form of planning and for your own sanity!
* Regularly reset the kernel and rerun the entire workbook. It is very easy to break something but not notice, because the correct version remains in memory.

## Marks
* __20 marks__: For many different _unit tests_ that will be run on `search`. They will cover all of the details in the above specification!
    * One test checks that it's faster than $0.1$ seconds on average (on the computer which runs the auto marker, which is quite fast) to do a search, so try not to be too inefficient (it ignores any time your notebook spends building data structures to be used by `search`). Note that the validation implementation comes in at $0.001$ seconds per search (after $5.5$ seconds of preparation), so this is very achievable!
    * You may want to look into Python's `set()` object, as it is useful for _one_ of the possible ways to implement this.
    * There will be sorting. The [Sorting how to](https://docs.python.org/3/howto/sorting.html) may help.
    * The auto marker does give some feedback, and you can run it as many times as you want. Don't be afraid to test an incomplete or semi-broken version of your code if stuck or unsure. It may help!
    * The validation implementation is 104 lines of code split over 5 cells (including white space for clarity and comments). Coded by someone who probably has much more experience than you, so you shouldn't aim to match this, but it's a good clue: If you find yourself at 500 lines you may want to stop and think some more! (line count does not include testing code, which is about the same amount again)

In [1]:
import pandas as pd
import numpy as np
import string
import re
import sys
from collections import Counter

######## Helper Functions ###########
def clean_list(l):
    """Delists items in a pandas series."""
    try:
        return ",".join(map(str, l)).replace(",", " ")
    except TypeError:
        return np.nan

def solution(emblem, user_entered):
    
    return not (Counter(emblem) - Counter(user_entered))
#####################################


def read_in_and_tokenise():
    """Reads in the json file and tokenises the data."""

    df = pd.read_json("recipes.json")

    # Only tokenise the first 4 columns
    for column in df.columns[:4]:

        # If the column contains a list, split up into a single string
        if column != "title":
            df.loc[:, f"{column}_tokens"] = df[column].apply(clean_list)

        # Remove Punctuation and Digits and make lowercase, Remove words < len(4)
        list_of_chars_and_digits = string.punctuation + string.digits
        df.loc[:, f"{column}_tokens"] = df[column].astype(str).str.replace(f"[{list_of_chars_and_digits}]", " ", regex=True).apply(str.lower).str.findall("\w{3,}").str.join(" ")

        # Split the tokens into individual words
        df.loc[:, f"{column}_tokens_list"] = df[f"{column}_tokens"].str.split(" ")

        # Count the occurrences
        df.loc[:, f"{column}_tokens_count"] = [Counter(i) for i in df[f'{column}_tokens_list']]

    # Remove duplicate tokens from 4 columns and store in one column TODO Add space padding to start and end for search
    df['tokens_set'] = (df['ingredients_tokens_list'] + df['title_tokens_list'] + df['directions_tokens_list'] + df['categories_tokens_list']).apply(set).apply(list).apply(clean_list)
    # df['tokens_set_list'] = df['tokens_set'].str.split(" ")
    
    # Remove connecting words - Doesn't save compute time
    '''
    word_count = Counter(word for row_list in df['tokens_set_list'] for word in set(row_list))
    remove_words = list({k: v for k, v in sorted(word_count.items(), key=lambda item: item[1], reverse=True)}.keys())[:15] # Slice of 15 most common words deleted, over 100k!
    
    df['tokens_reduced'] = [word for word in df['tokens_set_list'] if word not in remove_words]
    df.loc[:,'tokens_reduced'] = df['tokens_reduced'].apply(clean_list)
    '''
    
    return df


def count_occurrences(df, query):
    """Searches for the query and returns the counts."""
    
    # Search all four columns to find matches containing the query
    df['mask'] = True
    for word in query.split(' '):
        
        df.loc[:,'mask'] &= df["tokens_set"].str.contains(f' {word} ', regex=False)

    df = df[df['mask']].copy().reset_index(drop=True)

    # Create new columns which store how many times the query appears in each
    for column in df.columns[:4]:
        df.loc[:, f"{column}_count"] = [sum([df[f'{column}_tokens_count'][row].get(key, 0) for key in query.split(' ')]) for row in range(len(df))]

    return df


def scorer(df, count, ordering):
    """Scorer to filter the results of the search."""

    # Replace nans with zeros in the rating column
    #df.loc[:, "rating"] = df["rating"].fillna(0)

    if ordering == "normal":
        df.loc[:, "score"] = 8 * df["title_count"] + 4 * df["categories_count"] + 2 * df["ingredients_count"] + df["directions_count"] + df["rating"]

        # Sort by the score and return the 10 (or count) best values
        # Maximising
        df = df.sort_values(by=["score"], ascending=False).reset_index(drop=True).iloc[0:count:]

    elif ordering == "simple":
        # Check if there are nans in ingredients and directions, drop if so
        df = df.dropna(subset=["ingredients", "directions"]).copy()

        # Drop rows in len(1) as corrupted
        df = df[df["directions"].str.len() > 1]

        df.loc[:, "score"] = df["ingredients"].str.len() * (df["directions"].str.len())

        # Sort by the score and return the 10 (or count) best values
        # Minimising
        df = df.sort_values(by=["score"], ascending=True).reset_index(drop=True).iloc[0:count:]

    elif ordering == "healthy":
        # Check if there are nans in calories, protein or fat. Drop if so
        df = df.copy()
        df = df.dropna(subset=["calories", "protein", "fat"])
        
        calories = df["calories"]
        protein = df["protein"]
        fat = df["fat"]
        df.loc[:, "score"] = abs(calories - 510) / 510 + 2 * abs(protein - 18) / 18 + 4 * abs(fat - 150) / 150

        # Sort by the score and return the 10 (or count) best values
        # Minimising
        df = df.sort_values(by=["score"], ascending=True).reset_index(drop=True).iloc[0:count:]

    else:
        raise NotImplementedError(f"Ordering option {ordering} is not implemented.")

    # Print the titles of the best results
    results = list(df["title"])
    if len(results) > 0:
        print("\n".join(results))

    return df


In [2]:
df_read = read_in_and_tokenise()

def search(query, ordering="normal", count=10):
    """Assumes an input of keywords to search for seperated by a space."""

    # Remove all punctuation from input
    list_of_chars_and_digits = string.punctuation + string.digits
    query = pd.Series(query).str.replace(f"[{list_of_chars_and_digits}]", " ", regex=True).apply(str.lower)
    # Remove words of length 2 or 1
    query = query.str.findall("\w{3,}").str.join(" ")[0]
    # If empty string, return None
    if query == "":
        return

    df = count_occurrences(df_read, query)
    df = scorer(df, count, ordering)

    return df


In [3]:
results = search('pie', ordering='healthy', count=10)
results

Corn and Bacon Pie 
Spinach, Red Pepper, and Feta Quiche 
Zucchini, Sun-Dried Tomato, and Mozarella Tart 
Flaky Pie Crust Dough 
Chocolate Mousse Pie 
Bacon and Green Chili Quiche 
Yellow Squash and Mozzarella Quiche with Fresh Thyme 
Scallop and Shrimp Fritters with Chipotle Mayonnaise 
Butter Pie Crust 
Challah, Sausage, and Dried Cherry Stuffing 


,title,categories,ingredients,directions,rating,calories,protein,fat,title_tokens,title_tokens_list,...,directions_tokens,directions_tokens_list,directions_tokens_count,tokens_set,mask,title_count,categories_count,ingredients_count,directions_count,score
0,Corn and Bacon Pie,"[Milk/Cream, Egg, Brunch, Bake, Lunch, Bacon, ...","[1 cup unbleached all purpose flour, 3/4 cup f...",[Whisk first 3 ingredients in large bowl. Usin...,4.375,587.0,18.0,40.0,corn and bacon pie,"[corn, and, bacon, pie]",...,whisk first ingredients large bowl using back ...,"[whisk, first, ingredients, large, bowl, using...","{'whisk': 2, 'first': 1, 'ingredients': 2, 'la...",unsalted nonstick frozen ground add ingredient...,True,1,0,0,4,3.084314
1,"Spinach, Red Pepper, and Feta Quiche","[Egg, Brunch, Side, Bake, Vegetarian, Feta, Sp...","[1/3 cup plus 3 tablespoons all-purpose flour,...","[Preheat oven to 425°F., In a bowl with a past...",4.375,615.0,16.0,49.0,spinach red pepper and feta quiche,"[spinach, red, pepper, and, feta, quiche]",...,preheat oven bowl with pastry blender small fo...,"[preheat, oven, bowl, with, pastry, blender, s...","{'preheat': 1, 'oven': 3, 'bowl': 2, 'with': 3...",plus unsalted processor necessary add small ch...,True,0,0,0,1,3.121438
2,"Zucchini, Sun-Dried Tomato, and Mozarella Tart","[Milk/Cream, Egg, Tomato, Appetizer, Brunch, B...",[1 sheet frozen puff pastry (half of 17.3-ounc...,[Roll out pastry on floured surface to 1/8-inc...,4.375,538.0,17.0,38.0,zucchini sun dried tomato and mozarella tart,"[zucchini, sun, dried, tomato, and, mozarella,...",...,roll out pastry floured surface inch thick squ...,"[roll, out, pastry, floured, surface, inch, th...","{'roll': 1, 'out': 1, 'pastry': 4, 'floured': ...",plus frozen ground round small cheese package ...,True,0,0,0,1,3.152680
3,Flaky Pie Crust Dough,"[Dessert, Bake, Bon Appétit, Sugar Conscious, ...","[1 1/4 cups all purpose flour, 1/4 teaspoon sa...",[Whisk flour and salt in medium bowl to blend....,3.750,1530.0,17.0,110.0,flaky pie crust dough,"[flaky, pie, crust, dough]",...,whisk flour and salt medium bowl blend add but...,"[whisk, flour, and, salt, medium, bowl, blend,...","{'whisk': 1, 'flour': 1, 'and': 3, 'salt': 1, ...",unsalted add forms soften tablespoons added su...,True,1,0,0,0,3.177778
4,Chocolate Mousse Pie,"[Milk/Cream, Food Processor, Chocolate, Desser...",[21 chocolate sandwich cookies (such as Oreos)...,[Preheat oven to 350°F. Butter 9-inch-diameter...,5.000,1246.0,11.0,114.0,chocolate mousse pie,"[chocolate, mousse, pie]",...,preheat oven butter inch diameter springform p...,"[preheat, oven, butter, inch, diameter, spring...","{'preheat': 1, 'oven': 1, 'butter': 2, 'inch':...",optional unsalted processor process add small ...,True,1,0,0,0,3.180915
5,Bacon and Green Chili Quiche,"[Milk/Cream, Cheese, Dairy, Egg, Onion, Pork, ...",[1 refrigerated pie crust (half of15-ounce pac...,[Preheat oven to 425°F. Unfold crust. Using we...,4.375,714.0,20.0,54.0,bacon and green chili quiche,"[bacon, and, green, chili, quiche]",...,preheat oven unfold crust using wet fingertips...,"[preheat, oven, unfold, crust, using, wet, fin...","{'preheat': 1, 'oven': 2, 'unfold': 1, 'crust'...",knife dairy shape cheese package chilies quich...,True,0,0,1,1,3.182222
6,Yellow Squash and Mozzarella Quiche with Fresh...,"[Milk/Cream, Egg, Brunch, Bake, Mozzarella, Sq...","[1 1/4 cups unbleached all purpose flour, 1/2 ...",[Blend flour and salt in processor. Add butter...,4.375,660.0,20.0,49.0,yellow squash and mozzarella quiche with fresh...,"[yellow, squash, and, mozzarella, quiche, with...",...,blend flour and salt processor add butter usin...,"[blend, flour, and, salt, processor, add, butt...","{'blend': 3, 'flour': 1, 'and': 9, 'salt': 2, ...",crookneck unsalted processor ground add forms ...,True,0,0,0,1,3.209673
7,Scallop and Shrimp Fritters with Chipotle Mayo...,"[Shellfish, Fry, Summer, Gourmet]","[1/2 cup mayonnaise, 1 tablespoon finely chopp...","[Stir together mayonnaise, chil